<a href="https://colab.research.google.com/github/mahynski/pinn-library/blob/main/intro/schroedinger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PINN Example: Schroedinger Equation
===

This example is taken from Maziar Raissi's example [here](https://github.com/maziarraissi/PINNs/blob/master/main/continuous_time_inference%20(Schrodinger)/Schrodinger.py) which accompanies his original [work](https://www.sciencedirect.com/science/article/pii/S0021999118307125).